In [6]:
from typing import Any
import torch.nn as nn
import torch
from pytorch_lightning.utilities.types import STEP_OUTPUT
from torch.optim import Adam
import lightning as L

class LightningLSTM(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size = 1, hidden_size = 1)
    
    def forward(self, input):
        input_trans = input.view(len(input), 1)
        lstm_out, temp = self.lstm(input_trans)
        prediction = lstm_out[-1]

        return prediction
    
    def configure_optimizers(self):
        return Adam(self.parameters(), lr=0.1)
    
    def training_step(self, batch, batch_idx):
        input_i, label_i = batch
        output_i = self.forward(input_i[0])

        loss = (output_i - label_i)**2

        self.log("train_loss", loss)

        if label_i == 0:
            self.log('Out_0', output_i)
        else:
            self.log('Out_1', output_i)

        return loss
    

In [7]:
from torch.utils.data import TensorDataset, DataLoader

model = LightningLSTM()
input = torch.tensor([
    [0., 0.5, 0.25, 1.0],
    [1., 0.5, 0.25, 1.]
])
labels = torch.tensor([0., 1.])
dataset = TensorDataset(input, labels)
dataloader = DataLoader(dataset=dataset)
trainer = L.Trainer(max_epochs=300, log_every_n_steps=2)
trainer.fit(model=model, train_dataloaders=dataloader)

print("\n Now Lets compare the observed and predicted values")
print("Company A: Observed = 0, Predicted = 0 ", model(torch.tensor([0.,0.5,0.25, 1]).detach()))

print("\n Now Lets compare the observed and predicted values")
print("Company A: Observed = 1, Predicted = 0 ", model(torch.tensor([1.,0.5,0.25, 1]).detach()))

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type | Params
------------------------------
0 | lstm | LSTM | 16    
------------------------------
16        Trainable params
0         Non-trainable params
16        Total params
0.000     Total estimated model params size (MB)


Epoch 299: 100%|██████████| 2/2 [00:00<00:00, 62.50it/s, v_num=0] 

`Trainer.fit` stopped: `max_epochs=300` reached.


Epoch 299: 100%|██████████| 2/2 [00:00<00:00, 47.62it/s, v_num=0]

 Now Lets compare the observed and predicted values
Company A: Observed = 0, Predicted = 0  tensor([2.0444e-05], grad_fn=<SelectBackward0>)

 Now Lets compare the observed and predicted values
Company A: Observed = 1, Predicted = 0  tensor([0.9886], grad_fn=<SelectBackward0>)
